In [49]:
#!/usr/bin/env python3

import sys
sys.path.insert(0, '../pyaf/py_aff3ct/build/lib')

import numpy as np
import py_aff3ct as aff3ct
import math
import time
import matplotlib.pyplot as plt
from datetime import timedelta
from tempfile import TemporaryFile

from py_aff3ct.module.py_module import Py_Module
import py_aff3ct.tools.frozenbits_generator as tool_fb
import py_aff3ct.tools.noise as tool_noise
import py_aff3ct.tools.sparse_matrix as tool_sp
import random 

class source(Py_Module):
	# 
	# out : vec_cnt -> Où est-on dans le vecteur ? 
	# 		r_in	-> initial vector : never modified
	#		enable	-> Always true
	#		delta_x -> From -4 to 4
	#		delta_y -> From -4 to 4
	#		ix_x	-> Constant for now
	#		ix_y	-> Constant for now
	#		x		-> Position in the image along the X-axis
	#		y		-> Position in the image along the Y-axis
	#
	def generate(self):
		if (self.enable):
			self.x 			= self.vec_cnt // len(self.delta_y_range)
			self.y 			= self.vec_cnt %  len(self.delta_y_range)
			self.delta_x[:] = self.delta_x_range[self.x]	# Looks like : [[number]] with number = {-4 ; 4}
			self.delta_y[:] = self.delta_y_range[self.y]	# Looks like : [[number]] with number = {-4 ; 4}
			self.vec_cnt 	= self.vec_cnt + 1 	
			if(self.vec_cnt >= pow(len(self.delta_y_range),2)):
				self.enable = 0	# Tout les données ont été générées
		return 0
    
	def __init__(self, r_in, N): # N == len(r_in[0,:])

		Py_Module.__init__(self) # Call the aff3ct Py_Module __init__
		
		self.name   		= "source_xy"   		# Set your module's name
		self.r_in   		= r_in					# Constant
		self.delta_x_range 	= np.arange(-4,4,0.05)
		self.delta_y_range 	= np.arange(-4,4,0.05)
		self.enable 		= 1
		self.ix_x   		= np.random.randint(N)  # x position in the vector
		self.ix_y   		= self.ix_x + 5 % N		# y position in the vector
		self.x				= 0						# Position in the image along x
		self.y				= 0						# Position in the image along y

		self.vec_cnt 		= 0 					# counter for delta_x and delta_y
		self.delta_x 		= np.ndarray(shape = (1,len(self.delta_x_range)), dtype = np.float32)
		self.delta_y		= np.ndarray(shape = (1,len(self.delta_y_range)), dtype = np.float32)

		t_generate = self.create_task("generate") 	# create a task for your module

		s_r_in    = self.create_socket_out  (t_generate, "r_in"   , N		 , np.float32 ) # create an output socket for the task t_generate
		s_enable  = self.create_socket_out  (t_generate, "enable" , 1		 , np.int32   ) # create an output socket for the task t_generate
		s_delta_x = self.create_socket_out  (t_generate, "delta_x", 1		 , np.float32 )	# create an output socket for the task t_generate
		s_delta_y = self.create_socket_out  (t_generate, "delta_y", 1	 	 , np.float32 ) # create an output socket for the task t_generate
		s_ix_x    = self.create_socket_out  (t_generate, "ix_x"   , 1		 , np.int32   ) # create an output socket for the task t_generate
		s_ix_y    = self.create_socket_out  (t_generate, "ix_y"   , 1		 , np.int32   ) # create an output socket for the task t_generate
		s_x    	  = self.create_socket_out  (t_generate, "x"      , 1		 , np.int32   ) # create an output socket for the task t_generate
		s_y    	  = self.create_socket_out  (t_generate, "y"   	  , 1		 , np.int32   ) # create an output socket for the task t_generate

		self.create_codelet(t_generate, lambda slf, lsk, fid: slf.generate()) # create codelet

In [50]:
with open('noisy_cw.npy', 'rb') as f:
    r_in = np.load(f)

N = len(r_in[0,:])
src = source(r_in, N)

print(src['generate::r_in'][0,:]) 
""" 
    src['generate::r_in'][0,:] != r_in

    Pourquoi est-ce que le vecteur "r_in"
    devient nul alors que je ne lui 
    applique aucune modification ?
"""

# src['generate'].info()

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
- Task 
	- Name         : generate
	- Sockets      : 
	- Socket 0
		- Name              : r_in
		- Type              : out
		- Elements per frame: 128
		- Data type         : float32
	- Socket 1
		- Name              : enable
		- Type              : out
		- Elements per frame: 1
		- Data type         : int32
	- Socket 2
		- Name              : delta_x
		- Type              : out
		- Elements per frame: 1
		- Data type         : float32
	- Socket 3
		- Name              : delta_y
		- Type              : out
		- Elements per frame: 1
		- Data type         : float32
	- Socket 4
		- Name              : ix_